


#How to read data from the DSE into a notebook using Python


Firstly make sure that the default language is Python. You can change the default language in the dropdown above, next to Help. This sets the language for the notebook, but you can also change the language for each cell with the drop-down in the top right of the cell (_this_ cell is markdown)


###Data
A reminder: data in the DSE is stored in the catalog, which you can access from the menu on the left. Data is generally stored hierarchically in this order:

* Catalog (usually devcatalog), within which there are:
  * Schema - separated into the medallion architecture, and contain:
    * Tables
    * Views
    * Volumes

Tables and views are the way most structured data is stored - these are very similar to SQL tables you may be familiar with. They are essentially dataframes with defined types for each column.  Volumes are how we can store unstructured data and can contain basically anything - csv files, zip archives, etc. You can access both tables and volumes from within a notebook. 


The medallion architecture is a method of organizing schema:

 * 🥉 Bronze contains raw and unprocessed data - this may be a table or a store of csv files. 
 * 🥈 Silver layer data builds on the bronze layer - cleaned or reformatted bronze-level data for example. 
 * 🥇 Gold layer data is where the finished data output lives - this is reserved for clean and easily digestible data that can be piped to dashboards etc. 

You can add data to the DSE by going into the catalog and creating schema as needed - you can upload as a csv in the first instance, or scrape data using a notebook like this one. You could also create a volume and upload raw files, then process them yourself into a silver layer. You can change the permissions on the schema/tables to decide who can have access to the data. 


###Clusters
**For Python, you can use "serverless" when selecting the cluster, which will automatically provision resources as needed - this is the recommended practice, so you don't have to worry about clusters**  

Computational operations in the DSE are performed (generally speaking) on clusters - this is basically a virtual computer. You can choose/create a cluster using the dropdown above right - it will take a little while to spin up the first time it's created or switched on. The default settings are probably fine for your cluster, unless you have a need for large processing power or RAM. Remember that clusters are shared, so there _may_ be multiple users on a specific cluster, be mindful of this before you go restarting them willy-nilly. For a personal cluster, this is not much of a problem as you are the only user. Also, the cluster costs money all the time that it's enabled - so best practice is to turn it off if you're done with it and you know nobody else is going to use it. You don't need to be too parsimonious though - the cluster will turn itself off if it's idle for too long.




The DSE uses [Apache Spark](https://www.databricks.com/glossary/what-is-apache-spark) under-the-hood, which is a powerful technology that allows us to access and efficiently work with large datasets across clusters.  

To work with Spark in Python, we use PySpark - the Python API for Spark - which lets us interface directly with Spark. This is very powerful but it requires learning a new syntax - PySpark code




###Reading Data

As an example, we will read in data from the listsizes table, in the schema epd_silver, in the devcatalog. This is a table containing data on the list sizes (ie number of patients served) by GP practices. The data has already been converted from its raw monthly form, in the bronze layer, to a combined dataset in the silver layer 

The notation for specifying the table is very simple:

\<catalog\>.\<schema\>.\<table\>

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

catalog = "devcatalog"
schema = "epd_silver"
table = "listsizes"

data_tbl = spark.table(f"{catalog}.{schema}.{table}")

Easy! 

The data_tbl object that is returned is not a regular dataframe that you might be used to in pandas/polars - it's a Spark dataframe:

In [0]:
type(data_tbl)

This is a specific type of dataframe that is optimized for large datasets and clusters - it uses so-called [lazy-loading](https://en.wikipedia.org/wiki/Lazy_loading), meaning the table is not saved in memory and operations are not performed on the table until the user specifically requests them. This is good for large datasets, but it means that the familiar pandas functions you're used to may not work. A lot of them do however:

In [0]:
data_tbl.head(n=5)

...is fine, but there are Spark functions specifically suited to Spark dataframes that may work better. A useful one is display()  

In [0]:
display(data_tbl)

which looks prettier and has a download button. You can even click the little + to make a plot

You can use the collect() function in SparkR to convert the table from a Spark dataframe to a regular dataframe. This is generally fine for datasets that are not very large (so for anything with less than around a million rows or so)

In [0]:
regular_dataframe = data_tbl.toPandas()
type(regular_dataframe)

You can still use the display() function, which is handy, but now you can also use the familiar pandas operations

In [0]:
display(regular_dataframe.groupby("YEAR_MONTH")["total_patients"].sum().reset_index())

###Saving data

Saving data to the catalog in the DSE is easy. The command to save a dataframe df is:

In [0]:
df.write.mode("overwrite").saveAsTable("<catalog>.<schema>.<table>")

You can specify different modes - append, overwrite, etc. Be careful when using overwrite! We _can_ restore dropped tables, but we'd really rather not

There are some subtleties here in that this command only works for Spark dataframes, and not the pandas-compatible types with which we're all familiar. 
 
Fortunately it's very straightforward to change our pandas dataframe to Spark format before saving:



In [0]:
spark_df = spark.createDataFrame(regular_dataframe)
spark_df.write.mode("overwrite").saveAsTable("<catalog>.<schema>.<table>")